In [ ]:
import cv2
import numpy as np
import random

video = cv2.VideoCapture('data/Video_5.mp4')
texture = cv2.imread('data/Texture_2.jpg')
texture2 = cv2.imread('data/Texture_5.jpg')
texture3 = cv2.imread('data/Texture_6.jpg')
texture4 = cv2.imread('data/Texture_8.jpg')

frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
count = 0
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter('data/Video_10.mp4', fourcc, fps, (frame_width, frame_height), isColor=True)

img_threshold_type = cv2.THRESH_BINARY
adaptive_type = cv2.ADAPTIVE_THRESH_MEAN_C
adaptive_blocksize = 99
adaptive_C = 4

threshold_very_high = 200
threshold_low = 60
threshold_very_low = 25

resize_factor = 1

texture_height, texture_width, _ = texture.shape
texture2_height, texture2_width, _ = texture2.shape
texture3_height, texture3_width, _ = texture3.shape
texture4_height, texture4_width, _ = texture4.shape

while True:
    valid, img = video.read()
    
    if not valid:
        break
    
    texture_y, texture_x = 0, 0
    texture2_y, texture2_x = 0, 0
    texture3_y, texture3_x = 0, 0
    texture4_y, texture4_x = 0, 0
    
    small_img = cv2.resize(img, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    
    gray_img = cv2.cvtColor(small_img, cv2.COLOR_BGR2GRAY)
    
    binary_adaptive = cv2.adaptiveThreshold(gray_img, 255, adaptive_type, 
                                            img_threshold_type, adaptive_blocksize, adaptive_C)
    
    binary_adaptive_3_channel = cv2.cvtColor(binary_adaptive, cv2.COLOR_GRAY2BGR)
    
    white_mask = (binary_adaptive_3_channel == [255, 255, 255]).all(axis=2)
    
    for x in range(binary_adaptive_3_channel.shape[1]):
        for y in range(binary_adaptive_3_channel.shape[0]):
            if white_mask[y, x]:
                gray_value = gray_img[y, x]
                if gray_value < threshold_very_high and gray_value > threshold_low:
                    binary_adaptive_3_channel[y, x] = texture[texture_y, texture_x]
                    
                    texture_y += 1
                    if texture_y >= texture_height:
                        texture_y = 0
                        texture_x += 1
                        if texture_x >= texture_width:
                            texture_x = 0
                elif gray_value <= threshold_low and gray_value > threshold_very_low:
                    rand_x = random.randint(0, texture4.shape[1] - 1)
                    rand_y = random.randint(0, texture4.shape[0] - 1)
                    random_pixel_color = texture4[rand_y, rand_x]
                    binary_adaptive_3_channel[y, x] = random_pixel_color
                
                elif gray_value <= threshold_very_low:
                    binary_adaptive_3_channel[y, x] = texture3[texture3_y, texture3_x]
                    
                    texture3_y += 1
                    if texture3_y >= texture3_height:
                        texture3_y = 0
                        texture3_x += 1
                        if texture3_x >= texture3_width:
                            texture3_x = 0
                            
    for y in range(binary_adaptive_3_channel.shape[0]):
        for x in range(binary_adaptive_3_channel.shape[1]):
            if white_mask[y, x]:
                gray_value = gray_img[y, x]
                if gray_value >= threshold_very_high:
                    binary_adaptive_3_channel[y, x] = texture2[texture2_y, texture2_x]
                    
                    texture2_y += 1
                    if texture2_y >= texture2_height:
                        texture2_y = 0
                        texture2_x += 1
                        if texture2_x >= texture2_width:
                            texture2_x = 0
                            
    output.write(binary_adaptive_3_channel)
    count += 1
    print(f"{count} : {total_frames}")
                

video.release()
output.release()
cv2.destroyAllWindows()

print("end")